In [11]:
#load packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.interpolate import interp1d
import itertools as iter



In [12]:
#Tc_ss_list = np.arange(200,501,5)
#Tin_ss_list = np.arange(200,501,5)
#list(iter.product(Tc_ss_list,Tin_ss_list))
#Q_ss_list = [100]
#Cin_ss_list = [0.8]





In [13]:
# define CSTR model
def cstr(x,t,Tc, Cin, Q, Tin):
    # x contains:
    # Concentration of A in reactor (Ca)
    Ca = x[0]
    
    # and temperature in reactor (T)
    T = x[1]

    # Feed Temperature (K)
    Tf = Tin
    
    # Feed Concentration (mol/m^3)
    Caf = Cin
    
    # Parameters:
    # Volumetric Flowrate (m^3/sec)
    q = Q
    
    # Volume of CSTR (m^3)
    V = 100
    
    # Density of A-B Mixture (kg/m^3)
    rho = 1000
    
    # Heat capacity of A-B Mixture (J/kg-K)
    Cp = 0.239
    
    # Heat of reaction for A->B (J/mol)
    mdelH = 5e4
    
    # E - Activation energy in the Arrhenius Equation (J/mol)
    # R - Universal Gas Constant = 8.31451 J/mol-K
    EoverR = 8750
    
    # Pre-exponential factor (1/sec)
    k0 = 7.2e10
    
    # U - Overall Heat Transfer Coefficient (W/m^2-K)
    # A - Area - this value is specific for the U calculation (m^2)
    UA = 5e4
    
    # reaction rate
    rA = k0*np.exp(-EoverR/T)*Ca
    
    # Calculate concentration derivative
    dCadt = q/V*(Caf - Ca) - rA
    
    # Calculate temperature derivative
    dTdt = q/V*(Tf - T) \
            + mdelH/(rho*Cp)*rA \
            + UA/V/rho/Cp*(Tc-T)
    xdot = np.zeros(2)
    
    # it outputs dCa/dt and dT/dt
    xdot[0] = dCadt
    xdot[1] = dTdt
    return xdot

In [23]:
## Change levels here:

Tc_ss_list = [275,300]
Tin_ss_list = [300,325,350]
Q_ss_list = [100]
Cin_ss_list = [0.8]

combos = list(iter.product(Tc_ss_list,Tin_ss_list,Q_ss_list,Cin_ss_list))
df_all = pd.DataFrame()
for Tc_ss,Tin_ss, Q_ss, Cin_ss in combos: 
    for j in range(50):
        Tc_ss2 = Tc_ss + np.random.normal(0, 1, 1)
        #Tc_ss2 = Tc_ss
        #Tc_ss2 = Tc_ss2 + j*0.1
        #Tin_ss2 = Tin_ss + j*.1
        #Tin_ss2 = Tin_ss2 + j*0.1
        #Tin_ss2 = np.random.uniform(350,375,1)
        #Tc_ss2 = np.random.uniform(300,350,1)
        Tin_ss2 = Tin_ss + np.random.normal(0, 1, 1)
        #Tin_ss2 = Tin_ss
        #Q_ss = np.minimum(Q_ss + np.random.normal(0, 1, 1),100)
        #Cin_ss = np.minimum(Cin_ss + np.random.normal(0, 0.003, 1),1)
        #print(Tc_ss,Tin_ss,Q_ss, Cin_ss, j)
        print(Tin_ss2, Tc_ss2)
        
        # Steady State Initial Conditions for the States
        Ca_ss = 0.8
        T_ss = 350

        x0 = np.empty(2)
        x0[0] = Ca_ss
        x0[1] = T_ss

        # Steady State Initial Condition
        #Tc_ss = 275
        #Cin_ss = 1
        #Q_ss = 100
        #Tin_ss = 345


        # Time Interval (min)
        t = np.linspace(0,500,5001)



        t_level = np.linspace(0, 500, 51)

        T_level = np.ones(len(t_level)) * T_ss
        Tc_level = np.ones(len(t_level)) * Tc_ss2
        Cin_level = np.ones(len(t_level))*Cin_ss
        Q_level = np.ones(len(t_level))*Q_ss
        Tin_level = np.ones(len(t_level))*Tin_ss2


        Tc_linear = interp1d(t_level, Tc_level)
        #Tc_cubic = interp1d(t_level, Tc_level, kind='cubic') 
        Cin_linear = interp1d(t_level, Cin_level)
        #Cin_cubic = interp1d(t_level, Cin_level, kind='cubic') 
        Q_linear = interp1d(t_level, Q_level)
        #Q_cubic = interp1d(t_level, Q_level, kind='cubic')
        Tin_linear = interp1d(t_level, Tin_level)
        #Tin_cubic = interp1d(t_level, Tin_level, kind='cubic') 



        Tc = Tc_linear(t)
        Cin = Cin_linear(t)
        Q = Q_linear(t)
        Tin = Tin_linear(t)
        Ca = np.ones(len(t)) * Ca_ss
        T = np.ones(len(t)) * T_ss

        # Stepwise Cooling Temp
        #Tc[500:1000]  = 302
        #Tc[1000:1500] = 304
        #Tc[1500:2000] = 306
        #Tc[2000:2500] = 308
        #Tc[2500:3000] = 309
        #Tc[3000:3500] = 309.2
        #Tc[3500:4000] = 309.5
        #Tc[4000:4500] = 310
        #Tc[4500:]    = 300

        ## Change noise level here

        Tc = Tc + np.random.normal(0, 10, 5001)
        Cin = Cin + np.random.normal(0, .05, 5001)
        Q = Q + np.random.normal(0, 0.5, 5001)
        Tin = Tin + np.random.normal(0, 2, 5001)






    # Simulate CSTR

    #for i in range(1:18)
        for i in range(len(t)-1):
            ts = [t[i],t[i+1]]
            y = odeint(cstr,x0,ts,args=(Tc[i+1], Cin[i+1], Q[i+1], Tin[i+1]))
            Ca[i+1] = y[-1][0]
            T[i+1] = y[-1][1]
            x0[0] = Ca[i+1]
            x0[1] = T[i+1]
#             data = np.vstack((t,Tc,T)) # vertical stack
#             data = data.T              # transpose data
#             np.savetxt('cstr_step_tests_functional_ontology_linear_smooth.txt',data,delimiter=',',\
#                    header='Time,Tc,T',comments='')
        df = pd.DataFrame()

        df["time"] = t
        df["coolingTemp"] = Tc
        df["concentrationA_inlet"] = Cin
        df["Temp_inlet"] = Tin
        df["flowRate"] = Q
        df["concentrationA"] = Ca
        df["Temp"] = T
        df["Replicate"] = str(Tin_ss2) + "_" + str(Tc_ss2)+ "_"  +str(Q_ss)+"_"+str(j+1)
        
        df_all = pd.concat([df_all, df])


    

[372.91933882] [305.24478807]
[370.27239549] [334.38035674]
[351.61492118] [344.5672013]
[358.5686111] [316.76210368]
[361.98125491] [323.92046992]
[352.32923896] [322.55495688]
[363.45169735] [320.82100653]
[367.14644464] [304.7043181]
[351.11833917] [331.5589176]
[364.68128269] [335.04374525]
[358.88901881] [324.23353236]
[371.29136984] [338.82014375]
[367.21957623] [311.5077474]
[361.22819943] [328.07049585]
[365.79715271] [342.12113001]
[357.48565963] [347.47094482]
[374.07171224] [338.97029971]
[369.32813347] [308.51151747]
[365.37591747] [310.57348572]
[353.70314225] [338.59597384]
[351.839666] [318.89340818]
[353.76406763] [328.75325674]
[357.71516441] [313.02947827]
[351.3378517] [348.98469945]
[373.5231361] [303.7747957]
[365.45116252] [337.92691074]
[352.08590878] [344.55902545]
[373.78985692] [325.35467522]
[368.39185018] [332.06240202]
[356.28671979] [317.20697103]
[353.09534728] [342.08366708]
[357.74152953] [300.30690861]
[354.41881446] [315.98462176]
[355.31432289] [348.

In [24]:
df_all

,time,coolingTemp,concentrationA_inlet,Temp_inlet,flowRate,concentrationA,Temp,Replicate
0,0.0,304.702922,0.850038,374.855204,99.652845,0.800000,350.000000,[372.91933882]_[305.24478807]_100_1
1,0.1,303.827619,0.797626,373.740583,99.586477,0.682134,366.333905,[372.91933882]_[305.24478807]_100_1
2,0.2,301.406434,0.806182,371.278313,99.725924,0.000738,485.595536,[372.91933882]_[305.24478807]_100_1
3,0.3,308.253872,0.739771,373.422083,99.817867,0.002049,457.070470,[372.91933882]_[305.24478807]_100_1
4,0.4,302.482869,0.751742,373.954656,100.085323,0.005316,434.974322,[372.91933882]_[305.24478807]_100_1
...,...,...,...,...,...,...,...,...
4996,499.6,325.597132,0.744552,375.492924,99.320801,0.049546,391.080656,[372.25647096]_[325.54250503]_100_50
4997,499.7,323.129757,0.854618,376.271835,100.172014,0.056652,390.771368,[372.25647096]_[325.54250503]_100_50
4998,499.8,328.066460,0.749120,375.047777,100.315224,0.051420,391.609090,[372.25647096]_[325.54250503]_100_50
4999,499.9,326.593087,0.725490,373.531676,100.122263,0.049214,390.896141,[372.25647096]_[325.54250503]_100_50


In [22]:
df_all.to_csv("/mnt/DATA/testForSensSpec_stable_50runs_initialVariation.csv", index=False)

In [22]:
Q_ss = 99
Q_ss = np.minimum(Q_ss + np.random.normal(0, 1, 1),100)
Q_ss

array([99.68865354])

In [43]:
Tin_ss = 250
Tin_ss = Tin_ss + np.random.normal(0, 1, 50)
Tin_ss
#np.sort(Tin_ss)

array([249.58169234, 249.2764356 , 250.9014831 , 250.53491146,
       247.24490422, 250.18518779, 249.99687782, 247.88637479,
       251.05666039, 249.95550488, 250.82363871, 250.04908551,
       250.1050895 , 249.15069498, 249.54440943, 248.37926448,
       248.57502694, 251.71212915, 249.7920127 , 249.49903855,
       251.25438124, 250.71626289, 249.64352485, 249.42725682,
       249.91748181, 250.13888445, 250.42077372, 250.20237755,
       249.40871968, 252.04338656, 250.74364499, 251.35350004,
       249.96272355, 249.56791251, 249.3188457 , 251.79639193,
       250.75596828, 249.77923505, 248.75404613, 250.92779345,
       250.72673959, 249.49881196, 249.27581342, 248.90684699,
       249.93804575, 250.66091337, 250.8850215 , 250.20981969,
       250.20627837, 248.9358135 ])

In [58]:
np.arange(375,525,5)

array([375, 380, 385, 390, 395, 400, 405, 410, 415, 420, 425, 430, 435,
       440, 445, 450, 455, 460, 465, 470, 475, 480, 485, 490, 495, 500,
       505, 510, 515, 520])

In [9]:
Tc_ss_list = [300,350]
Tin_ss_list = [340,345]
Q_ss_list = [100,105]
Cin_ss_list = [0.8,0.9]

combos = list(iter.product(Tc_ss_list,Tin_ss_list,Q_ss_list,Cin_ss_list))